### Tutorial covering basic concepts of using Sqlite with Python
- Create Table
- Alter Table
- Insert Values into the table
- Print table
- Drop and alter table
- Running Queries using execute command
- Null handling with Sqlite in python
- SQL injection
- Finding Errors
- User defined SQL functions
- Some Meta data (row_factory)



In [1]:
#create table statements
import sqlite3
dbPath="./register_concepts.db"
conn = sqlite3.connect(dbPath)
c = conn.cursor()
c.execute(' PRAGMA foreign_keys=ON; ')
c.execute("DROP table if exists department;")
c.execute('''CREATE TABLE department (dept_name TEXT, dept_id INTEGER, PRIMARY KEY (dept_id));''')
c.execute('''ALTER TABLE department ADD year INTEGER;''')
c.execute('''ALTER TABLE department ADD num_faculty INTEGER; ''')
conn.commit()
# Inserting values into the table
c.execute('''INSERT INTO department VALUES            
			('Computing Science', 1, 2000, 100),            
			('Electrical', 2, 2008, 152),            
			('Mechanical', 3, 2016, 108);            
		''')
conn.commit()

In [2]:
#used to print the table
def printTable(t):
    try:
        c.execute('select * from %s;'%(t,))
        tRows=c.fetchall()
        for row in tRows:
            print(row)
    except:
        print("couldn't print %s, are you sure it exists?"%(t,))
        
printTable("department")

('Computing Science', 1, 2000, 100)
('Electrical', 2, 2008, 152)
('Mechanical', 3, 2016, 108)


In [3]:
#execute() will raise warnings if it is used for multiple queries at the same time
c.execute("DROP table if exists temp;")
c.executescript('''ALTER TABLE department RENAME TO temp;
                    CREATE TABLE department (
                    dept_name TEXT,
                    dept_id INTEGER,
                    faculty_count INTEGER, year INTEGER, 
                    PRIMARY KEY (dept_id));
                    INSERT INTO department SELECT dept_name, dept_id, num_faculty , year  FROM temp;
                    DROP TABLE temp;''')
conn.commit()
printTable("department")

('Computing Science', 1, 100, 2000)
('Electrical', 2, 152, 2008)
('Mechanical', 3, 108, 2016)


In [4]:
#Queries using execute command
#Question mark placeholder method
dept_number=(1,)
c.execute('SELECT * FROM department WHERE dept_id=? ;', dept_number)
#Named-placeholder method
dept_number=1
dept_year=2000
c.execute("SELECT * FROM department WHERE dept_id=:num and year=:year",   {"num":dept_number, "year": dept_year})

#Rerieve results using fetchone command
row=c.fetchone()
print (row[0])
c.execute("SELECT * FROM department;")
rows=c.fetchall()
print (rows)
insertions = [('Chemistry', 4, 200, 2002),       
('Law', 5, 160, 2010),  ('Business', 6, 208, 2018)]
c.executemany(" INSERT INTO department VALUES (?,?,?,?) ", insertions);
conn.commit()
printTable("department")







Computing Science
[('Computing Science', 1, 100, 2000), ('Electrical', 2, 152, 2008), ('Mechanical', 3, 108, 2016)]
('Computing Science', 1, 100, 2000)
('Electrical', 2, 152, 2008)
('Mechanical', 3, 108, 2016)
('Chemistry', 4, 200, 2002)
('Law', 5, 160, 2010)
('Business', 6, 208, 2018)


In [5]:
conn.row_factory = sqlite3.Row
c = conn.cursor()
c.execute("SELECT * FROM department;")
row = c.fetchone()
print (row.keys()) 
rows = c.fetchall()
for each in rows:    
    print (each["dept_name"], each["year"])


['dept_name', 'dept_id', 'faculty_count', 'year']
Electrical 2008
Mechanical 2016
Chemistry 2002
Law 2010
Business 2018


In [6]:
#Null Handling
c.execute("INSERT INTO department VALUES ('Civil',7,120,2000), ('Biology',8,128,2016), ('Design',9,null,null);")
conn.commit()
#To only return full defined record:
c.execute("SELECT * FROM department WHERE year IS NOT NULL;")
#To return records that contain null(s):
c.execute("SELECT * FROM department WHERE year IS NULL;")
conn.commit()
            

In [7]:
#Python NONE data type ≈ SQLite NULL
c.execute('SELECT * FROM department;')
depts = c.fetchall()
for dept in depts:
    for year in dept:
        if year == None:
            print(dept)
            break
            
#conn.close()

In [8]:
#insert different values for its parameters through a single query
insertions = [	('Education', 10, 200, 2002),       
			('Physics', 11, 160, 2010),
            ('Psychology', 12, 208, 2018),]
c.executemany(" INSERT INTO department VALUES (?,?,?,?) ", insertions);
conn.commit();
printTable("department");

In [9]:
#create table statements
import sqlite3
conn = sqlite3.connect(dbPath)
c = conn.cursor()
c.execute(' PRAGMA foreign_keys=ON; ')
c.execute("DROP table if exists Account;")
c.execute('''CREATE TABLE Account (username TEXT, password TEXT, name TEXT, phone TEXT, birthday INTEGER, PRIMARY KEY (username));''')
conn.commit()
c.execute('''INSERT INTO Account VALUES    
			('root', 'root123', 'admin', '5554449898', 1991), 
			('John_Mo', 'J1234', 'John', '5554443232', 1990),            
			('Anna_Lu', 'A1234', 'Anna', '5554447373', 1992),            
			('Emma_Bo', 'E1234', 'Emma', '5554448787', 1995);            
		''')
conn.commit()
printTable("Account")


('root', 'root123', 'admin', '5554449898', 1991)
('John_Mo', 'J1234', 'John', '5554443232', 1990)
('Anna_Lu', 'A1234', 'Anna', '5554447373', 1992)
('Emma_Bo', 'E1234', 'Emma', '5554448787', 1995)


In [10]:
username = "root"
password = "root123"
c.execute("SELECT * FROM Account WHERE username='%s' and password='%s' ;" % (username, password))
accounts = c.fetchall()
for account in accounts:
    print(account)
    break

('root', 'root123', 'admin', '5554449898', 1991)


In [11]:
#attacker!!
username = "root"
password = "attacker' or '1' = '1"
c.execute("SELECT * FROM Account WHERE username='%s' and password='%s' ;" % (username, password))
accounts = c.fetchall()
for account in accounts:
    print(account)
    break

('root', 'root123', 'admin', '5554449898', 1991)


In [12]:
#prevent SQL injections solution 1
print ("**Real User**")
username = "root"
password = "root123"
c.execute("SELECT * FROM Account WHERE username=:user and password=:pass ;" , {"user": username, "pass":password})
accounts = c.fetchall()
for account in accounts:
    print(account)
    break

print ("**Attacker**")   
username = "root"
password = "attacker' or '1' = '1"
c.execute("SELECT * FROM Account WHERE username=:user and password=:pass ;" , {"user": username, "pass":password})
accounts = c.fetchall()
for account in accounts:
    print(account)
    break

**Real User**
('root', 'root123', 'admin', '5554449898', 1991)
**Attacker**


In [13]:
#prevent SQL injections solution 2
print ("**Real User**")
username = "root"
password = "root123"
c.execute("SELECT * FROM Account WHERE username=? and password=? ;" , (username, password))
accounts = c.fetchall()
for account in accounts:
    print(account)
    break

print ("**Attacker**")   
username = "root"
password = "attacker' or '1' = '1"
c.execute("SELECT * FROM Account WHERE username=? and password=? ;" , (username, password))
accounts = c.fetchall()
for account in accounts:
    print(account)
    break


**Real User**
('root', 'root123', 'admin', '5554449898', 1991)
**Attacker**


In [14]:
#prevent SQL injections solution 3
import re

print ("**Real User**")
username = "root"
password = "root123"
if re.match("^[A-Za-z0-9_]*$", username) and re.match("^[A-Za-z0-9_]*$", password):
	c.execute('SELECT * FROM Account WHERE username=? and password=?;' , (username, password))
accounts = c.fetchall()
for account in accounts:
    print(account)
    break
    
print ("**Attacker**")   
username = "root"
password = "attacker' or '1' = '1"
if re.match("^[A-Za-z0-9_]*$", username) and re.match("^[A-Za-z0-9_]*$", password):
	c.execute('SELECT * FROM Account WHERE username=? and password=?;' , (username, password))
accounts = c.fetchall()
for account in accounts:
    print(account)
    break

**Real User**
('root', 'root123', 'admin', '5554449898', 1991)
**Attacker**


In [15]:
#Finding Errors
text = "INSERT INTO Account VALUES ('test_complete', 'T123', 'SemiColoumn', '5554446161', 1990);"
if sqlite3.complete_statement(text):
	c.execute(text)

text = "INSERT INTO Account VALUES ('test_incomplete', 'T123', 'NoSemiColoumn', '5554446262', 1990)"
if sqlite3.complete_statement(text):
	c.execute(text)
    
conn.commit()
printTable("Account")

('root', 'root123', 'admin', '5554449898', 1991)
('John_Mo', 'J1234', 'John', '5554443232', 1990)
('Anna_Lu', 'A1234', 'Anna', '5554447373', 1992)
('Emma_Bo', 'E1234', 'Emma', '5554448787', 1995)
('test_complete', 'T123', 'SemiColoumn', '5554446161', 1990)


In [16]:
#Try-catching for errors:

text = "INSERT INTO Account VALUES ('John_Mo', 'JJ12', 'John', '5554440101', 1994);"
try: 
	      c.execute(text)
except sqlite3.Error as e:
    print ('Error:', e.args[0])

Error: UNIQUE constraint failed: Account.username


In [17]:
#Encrypt Passwords

import hashlib
def encrypt(password):
	alg = hashlib.sha256() 
	alg.update(password.encode("utf-8"))
	return alg.hexdigest()
	
conn.create_function("hash", 1, encrypt) 
data = ("Robin_Na", "R1234", "Robin", "5554449797", 1997);
c.execute(" INSERT INTO Account (username, password, name, phone, birthday) VALUES (?, hash(?), ?, ?, ?) ", data );
conn.commit()
printTable("Account")

('root', 'root123', 'admin', '5554449898', 1991)
('John_Mo', 'J1234', 'John', '5554443232', 1990)
('Anna_Lu', 'A1234', 'Anna', '5554447373', 1992)
('Emma_Bo', 'E1234', 'Emma', '5554448787', 1995)
('test_complete', 'T123', 'SemiColoumn', '5554446161', 1990)
('Robin_Na', 'f6f4a68ffee3946897fd98ace4798fd7c45d4d43d126199efc7069cc714a65b5', 'Robin', '5554449797', 1997)


In [18]:
#Check Passwords

import hashlib
def encrypt(password):
	alg = hashlib.sha256() 
	alg.update(password.encode("utf-8"))
	return alg.hexdigest()

conn.create_function("hash", 1, encrypt) 

data = ("R1234", )
c.execute(" SELECT phone FROM Account WHERE password LIKE hash(?) ", data);
phones = c.fetchall()
for phone in phones:
    print(phone)
    break

('5554449797',)


In [19]:
c.execute(" SELECT * from Account; ")
print ("name of the first column: " + c.description[0][0])


name of the first column: username


In [20]:
#Row Factory

def dictionary_factory (cursor, row):
	dict = {}
	for i, col in enumerate(cursor.description): 
		dict[col[0]] = row[i]
	return dict
  
conn.row_factory = dictionary_factory 
cur = conn.cursor()

cur.execute(" SELECT * from Account; ")
result = cur.fetchone()
print (" the first column: ")
print ( result['username'] )


 the first column: 
root
